In [2]:
import os

import sqlite3
import pandas as pd

In [3]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects','datasci')
dthon_db = os.path.join(dthon_path, "melbourne-datathon","data","medi.db")
# Run this first to create database connection
conn = sqlite3.connect(dthon_db)

In [ ]:
# Anthonies query
sql = """
SELECT t1.Patient_ID, t1.Drug_ID, t2.Drug_ID
FROM transactions_depression_vic t1
JOIN transactions_depression_vic t2
WHERE t1.Patient_ID = t2.Patient_ID
AND t1.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID = GenericIngredientName
)
AND t2.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID IN (
        SELECT GenericIngredientName
        FROM Drug_LookUp
        WHERE MasterProductID = t1.Drug_ID
    )
    WHERE GenericIngredientName = ai.DrugB_ID
)
"""

%time df = pd.read_sql_query(sql, conn)
df.head()

In [4]:
# Anthonies query
sql = """
CREATE TEMP TABLE transactions_depression
AS
SELECT * FROM transactions t
WHERE t.Drug_ID IN (
    SELECT c.MasterProductID
    FROM ChronicIllness_LookUp c
    WHERE c.ChronicIllness = 'Depression'
)
"""

%time df = pd.read_sql_query(sql, conn)

TypeError: 'NoneType' object is not iterable

In [6]:
# Anthonies query
sql = """
CREATE TEMP TABLE transactions_depression_vic
AS
SELECT * FROM transactions_depression t
WHERE t.Patient_ID IN (
    SELECT Patient_ID
    FROM patients p
    WHERE p.postcode IN (
        SELECT postcode FROM postcodes_geo
        WHERE state = 'VIC'
    )
)
"""
%time pd.read_sql_query(sql, conn)

TypeError: 'NoneType' object is not iterable